In [1]:
import re
import joblib

import numpy as np
import pandas as pd

import seaborn as sns

import tensorflow as tf

import matplotlib.pyplot as plt

import tensorflow_recommenders as tfrs


from typing import Dict, Text
from datetime import datetime
from itertools import combinations

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

In [2]:
users_df = pd.read_csv("../data/users.csv")
users_df.head()

,user_id,user_name,user_gender,user_birth_date,user_age,user_location,user_tags,user_preferences
0,ba01ce05-a24e-4fc6-8c53-21b1faa624a9,Manuella Beatrice,P,5/27/2002,21,"Banten, Serang",Bernyanyi|Independen|Membaca|Disiplin|Menulis,Ingin memiliki teman yang bisa diajak kulinera...
1,ebeac7d2-642f-42a6-9175-6eb86dbcf84d,Achmad Irvan Hakim,L,10/21/2003,20,"Kalimantan Timur, Bontang",Pecinta alam|Membaca|Kritis|Memasak|Ngopi,"Teman yang enak di ajak ngapa ngaapin (mabar,j..."
2,25fe48f2-a312-421d-8751-984b5e4a448f,Lubna Mawaddah,P,1/12/2003,20,"Jawa Tengah, Semarang",Puzzle|Bernyanyi|Penerjemah|Kecerdasan buatan|...,"Saya ingin teman yang bisa diajak kulineran, k..."
3,f5b01cb5-d06c-41fc-880a-694d9c74e96c,Fajar Ramadhan,L,10/27/2005,18,"Jawa Barat, Tasikmalaya",Olahraga|Analis keamanan|Memanjat|Binaraga|Han...,"Temen yang bisa diajak belajar, diskusi terkai..."
4,b8307662-eb4f-4222-845e-771736274160,Gede Putra Nugraha,L,6/19/2002,21,"Jawa Barat, Cirebon",Dokter|Analitis|Kuliner|Musisi|Sosmed,Saya ingin teman yang bisa diajak berkembang b...


In [3]:
talents_df = pd.read_csv("../data/talents.csv")
talents_df.head()

,talent_id,talent_name,talent_gender,talent_birth_date,talent_age,talent_location,talent_tags,talent_description
0,fc0f6e4b-c397-40a4-a24c-d08b92aedc45,Pribadi Anwar Wicaksono,L,24/4/1992,31,"Banten, Cilegon",Badminton|Berjiwa petualang|Kpopers|Hiking|Ola...,"Saya sangat menyukai Kpop, girlband favorit sa..."
1,674c14d8-ed13-471a-aabf-ceef4e5bc44b,Perdana Anshari,L,10/3/2005,18,"Kalimantan Utara, Tarakan",Desain|Inovatif|Optimis|Seniman|Kreatif,Saya adalah seorang seniman visual yang mencin...
2,f2427a83-eb72-47dd-aeaf-cf131b5cefa2,Dwi Nur Septasya Nahda,P,10/3/2002,21,"Sumatera Selatan, Pagaralam",Petualangan|Bersepeda|Kutu buku|Traveling|Kuliner,Pecinta kuliner dan petualangan! Saya suka men...
3,ac56a04d-aa98-4591-8e8c-2b188040c114,Asmara Tamara Liranda,P,31/3/1992,31,"Sumatera Barat, Lima Puluh Kota",Analitis|Coding|Inovatif|Teknologi|Programmer,Seorang pengembang perangkat lunak yang bersem...
4,d7a31753-7f7a-4d3a-80ec-bca26e578735,Adisyafitri Oky Septasya,P,12/7/1990,33,"Kalimantan Timur, Samarinda",Ambisius|Membaca|Cerdas|Pengajar|Visioner,Pengajar bahasa Inggris yang juga pencinta lit...


In [4]:
reviews_df = pd.read_csv("../data/reviews.csv")
reviews_df.head()

,user_id,talent_id,rating,review
0,5de6b455-0a80-4047-90e7-8ddee7ee2bb8,6742ab46-c8ca-4acc-99b2-9ba1c5bfccb6,4,"Talent ini sangat kreatif dalam fotografi, has..."
1,5de6b455-0a80-4047-90e7-8ddee7ee2bb8,d9b90682-f5b4-45f7-b1ed-8b8044c945e1,3,"Kemampuan olahraga talent ini memuaskan, tetap..."
2,bc92f639-6659-4732-ad42-cc62b377f811,1b18b89b-5434-422d-b2d9-294e4c64657e,5,Sangat puas dengan pengalaman kuliner bersama ...
3,9a8427dd-ef41-46d9-86fe-063129e8190e,931c2814-1f84-404f-851a-7fb075835d95,4,Talent ini memberikan konsultasi bisnis yang s...
4,0e9ced1a-35a0-4ad5-a204-3d5c843de78f,ddc39acb-d29b-43f0-abcd-f54fe7be2904,2,Kemampuan gym dan jogging talent ini tidak ses...


In [5]:
vectorizer_tags = TfidfVectorizer()
vectorizer_tags.fit(talents_df["talent_tags"])

joblib.dump(vectorizer_tags, "../models/vectorizer_tags.joblib")

['../models/vectorizer_tags.joblib']

In [6]:
vectorizer_text = TfidfVectorizer()
vectorizer_text.fit(talents_df["talent_description"])

joblib.dump(vectorizer_text, "../models/vectorizer_text.joblib")

['../models/vectorizer_text.joblib']

In [7]:
vectorizer_tags = joblib.load('../models/vectorizer_tags.joblib')
talent_tags_tfidf = vectorizer_tags.transform(talents_df['talent_tags'])

In [8]:
vectorizer_text = joblib.load("../models/vectorizer_text.joblib")
talent_text_tfidf = vectorizer_text.transform(talents_df["talent_description"])

In [9]:
new_user_tags = "Desain|Fashion|Kreatif|Perfeksionis|Ramah"
new_user_preferences = "Ingin memiliki teman yang bisa diajak kulineran dan staycation untuk mengelilingi Indonesia"

new_user_tags_tfidf = vectorizer_tags.transform([new_user_tags])
new_user_text_tfidf = vectorizer_text.transform([new_user_preferences])

cosine_similarities_tags = linear_kernel(new_user_tags_tfidf, talent_tags_tfidf)
cosine_similarities_text = linear_kernel(new_user_text_tfidf, talent_text_tfidf)

cosine_similarities_combined = (
    0.5 * cosine_similarities_tags + 0.5 * cosine_similarities_text
)

# Get the indices of talents sorted by similarity
talent_indices = cosine_similarities_combined.argsort()[0][::-1]

num_recommendations = 100
recommended_talents = [talents_df.loc[i, "talent_id"] for i in talent_indices][
    :num_recommendations
]

print(f"Recommendations for the New User: {recommended_talents}")

Recommendations for the New User: ['28476cd6-1f0e-4117-acc4-cc61c2d806a4', '20c8f636-0405-4f64-a34f-a2b53db0f30b', '1f17bf3f-dc60-4584-97d0-5f66d3c127e0', '79acdf8c-54cd-48d2-9dd1-52de35b9be29', 'c21a169a-912b-4362-879c-ba4f698409b8', '484ee130-c1ee-4fa3-ace3-f50ca80443b3', 'af738bff-52f3-4358-b553-fc550953725c', '61601bd4-79b7-4399-b6b3-4bad407c2e06', '197a7fe4-ca2e-430d-bf21-dc3929565e0e', '63033218-389c-4377-854c-dd415337b623', '8567b129-dcc8-47f0-be27-3584d77dea1d', 'facf35ed-13fd-4ce1-b09c-5e4c97d8f95c', '1f73feb0-74d1-40b5-9b5e-645c84651199', '2ae4c607-2a81-42d8-8093-8c87aab013b9', '7ff10a48-0f1a-40af-93c8-6960d345309a', 'bd5d45ec-717f-459e-a1ae-2d82bbb8e3b0', '34b384a4-cbb9-4e39-9f85-8b80423df524', '60c22d64-ec40-43bd-927a-18a969bb8584', 'ca0ad2c1-0d38-484b-94d8-c09a2f23d6a5', '5914ab27-c5d8-4a91-be53-598bb190794e', 'e296c826-23c7-43b0-948a-ce88727a8816', '478ad7cd-8db6-4bb8-a202-c94d5a3fc25d', '18e716ea-264e-4029-bf64-786456417d18', '239e6b47-80bb-41ae-8297-b0d7492ed472', 'fe72

In [10]:
new_user_tags_tfidf.shape, new_user_text_tfidf.shape

((1, 218), (1, 1368))

In [11]:
cosine_similarities_tags.shape, cosine_similarities_text.shape

((1, 509), (1, 509))

In [12]:
talents_df.loc[talents_df.talent_id.isin(recommended_talents), :].head()

,talent_id,talent_name,talent_gender,talent_birth_date,talent_age,talent_location,talent_tags,talent_description
1,674c14d8-ed13-471a-aabf-ceef4e5bc44b,Perdana Anshari,L,10/3/2005,18,"Kalimantan Utara, Tarakan",Desain|Inovatif|Optimis|Seniman|Kreatif,Saya adalah seorang seniman visual yang mencin...
12,64183f98-8491-43e5-bcec-b057ae943a57,Andria Sri,P,25/4/1997,26,"Kalimantan Tengah, Palangka Raya",Desain|Cerdas|Pengajar|Kreatif|Seni,Seorang seniman kreatif yang mahir dalam seni ...
14,1b34b365-c0bc-48d9-a66f-042ba1cf090d,Haq Deni Haswan,L,18/3/2000,23,"Gorontalo, Gorontalo",Petualangan|Fashion|Filsafat|Traveling|Fotografi,Seorang fotografer pecinta alam yang gemar mel...
33,34b384a4-cbb9-4e39-9f85-8b80423df524,Novita Melinda,P,19/11/1991,32,"Sumatera Utara, Pematangsiantar",Desain|Inovatif|Menggambar|Kreatif|Psikolog,Seorang desainer grafis yang memiliki keahlian...
35,48645e45-09b3-43cc-a42d-01eefd058880,Rahmadi Hedy Pratama,L,6/7/1990,33,"Kalimantan Tengah, Kotawaringin Timur",Analitis|Desain|Kreatif|Fotografer|Seni,Seorang seniman digital yang ahli dalam seni r...


In [13]:
talents_df.loc[(talents_df.talent_id.isin(recommended_talents)) & (talents_df.talent_location.str.contains('Jawa Timur', case=False)), :]

,talent_id,talent_name,talent_gender,talent_birth_date,talent_age,talent_location,talent_tags,talent_description
45,c3318550-6af5-42a2-86b3-08498b54c652,Restiantoro Nainggolan,L,15/9/1998,25,"Jawa Timur, Mojokerto",Sabar|Membaca|Filsafat|Kreatif|Model,Seorang desainer grafis yang memiliki keahlian...
83,197a7fe4-ca2e-430d-bf21-dc3929565e0e,Yansyah Aljufrie Restiantoro Tenrian,L,26/7/2002,21,"Jawa Timur, Surabaya",Desain|Ambisius|Fashion|Arkeolog|Kreatif,Perancang mode yang menciptakan desain busana ...
115,fd4126a9-6bee-48ec-b010-6da5c2179937,Muhammad Muhammad,L,4/4/1998,25,"Jawa Timur, Pasuruan",Diskusi|Desain|Sabar|Pengajar|Disiplin,Guru biologi yang berkomitmen pada pendidikan ...
149,2a335d19-d04e-454f-baf6-2399420847fe,Helvy Dwini Inka Seftiara,P,14/5/1984,39,"Jawa Timur, Malang",Analitis|Coding|Inovatif|Programmer|Perfeksionis,Pengembang backend dengan keahlian dalam Node....
190,2ae4c607-2a81-42d8-8093-8c87aab013b9,Renaldy Rizal Idzharul Osama,L,12/5/1983,40,"Jawa Timur, Blitar",Fashion|Patriotik|Kreatif|Skena|Ramah,Pecinta fashion yang juga memiliki ketertarika...
192,167c23a2-5f3d-48f1-87a5-e7e8b4ce8e44,Sayid Bryan Karim Restiantoro,L,26/8/1978,45,"Jawa Timur, Mojokerto",Penyanyi|Musisi|Kreatif|Psikolog|Ramah,Pecinta musik yang memiliki kecintaan pada ber...
371,239e6b47-80bb-41ae-8297-b0d7492ed472,Zahra Wahyuningrum Aprilia,P,15/5/1982,41,"Jawa Timur, Malang",Desain|Coding|Humoris|Kreatif|Programmer,Seorang pengembang front-end yang berfokus pad...
389,8fae7796-c218-438c-8997-0b02453d54ca,Dwi Faradila Amalanda,P,27/5/1990,33,"Jawa Timur, Mojokerto",Desain|Kreatif|Terorganisir|Arsitek|Seni,Seorang perancang taman yang menciptakan ruang...
394,bd5d45ec-717f-459e-a1ae-2d82bbb8e3b0,Chairi Fadlil Liwal,L,16/5/2000,23,"Jawa Timur, Kediri",Desain|Seni|Kreatif|Ramah|Intelektual,Seorang desainer kemasan yang menciptakan desa...
429,ca0ad2c1-0d38-484b-94d8-c09a2f23d6a5,Nur Puspita,P,25/1/1979,44,"Jawa Timur, Blitar",Desain|Coding|Teknologi|Kreatif|Programmer,Seorang pengembang front-end yang ahli dalam m...
